## GNEO Axis 3 Hub Resource Catalogue demo

In this part of the demo, the user will use the system level resource catalogue endpoint to discover data collections and datasets.
The `owslib.csw` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://catalogue.{base_domain}/csw'

In [ ]:
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

Service metadata shown here includes identification type (from ISO-19115), CSW version and supported operations

In [ ]:
csw.identification.type

In [ ]:
csw.version

In [ ]:
[op.name for op in csw.operations]

As well as catalogue queryables:

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

The user can make a GetRecords request to get all records of the catalogue, with a page limit of 10.

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
   print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

If the user wishes to discover data with usage of filters, an OGC Filter can be used. Here we demonstrate how to create spatial (`bbox`), temporal (`time`), and attribute (`apiso:CloudCover`) filters combined with logical operators like and/or

In [ ]:
bbox_query = BBox([37.8, 23.3, 38.8, 24.5])
#bbox_query = BBox([39.66, 19.82, 40.64, 21.11])
# bbox_query = BBox([37, 13.9, 37.9, 15.1])
# bbox_query = BBox([47.7, 14.9, 48.7, 16.4])

In [ ]:
begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2024-11-20 00:00')

In [ ]:
end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2024-11-21 00:00')

In [ ]:
cloud = PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            begin, end,  # start and end date
            cloud        # cloud
        ]
    )
]

The filter is then applied to the GetRecords request and results are shown:

In [ ]:
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
selected_record = list(csw.records)[0]

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification[0].identtype}\ntitle: {csw.records[rec].identification[0].title}\n')

Another option is to perform a collection level search, using the `apiso:parentIdentifier` queryable. Here only the Sentinel2 L1C datasets will be discovered.

In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'sentinel-2-l2a')

In [ ]:
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

Or we can just search using the bbox filter

In [ ]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
for rec in csw.records:
   print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification[0].identtype}\ntitle: {csw.records[rec].identification[0].title}\n')

Or we can perform a full text search (here the keyword Orthoimagery is used)

In [ ]:
# anytext_query = PropertyIsEqualTo('csw:AnyText', 'Orthoimagery')
anytext_query = PropertyIsEqualTo('csw:AnyText', 'msi')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            anytext_query # any text
        ]
    )
]

In [ ]:
csw.getrecords2(constraints=filter_list)
csw.results

We can also iterate through the catalogue search results by passing the `startposition` and `maxrecords` parameters to GetRecords request:

In [ ]:
# csw_records = {}
# sortby = SortBy([SortProperty('dc:title', 'ASC')])
# pagesize=10
# maxrecords=1000
# startposition = 0
# nextrecord = getattr(csw, 'results', 1)
# while nextrecord != 0:
#    csw.getrecords2(constraints=[anytext_query], startposition=startposition,
#                    maxrecords=pagesize, sortby=sortby)
#    csw_records.update(csw.records)
#    if csw.results['nextrecord'] == 0:
#        break
#    startposition += pagesize
#    if startposition >= maxrecords:
#        break
# csw.records.update(csw_records)
# records = '\n'.join(csw.records.keys())
# print('Found {} records.\n'.format(len(csw.records.keys())))
# for key, value in list(csw.records.items()):
#    print(f'identifier: {value.identifier}\ntype: {value.type}\ntitle: {value.title}\n')

The user then selects a record identifier and asks the catalogue to fetch the full record. Here we demonstrate how to obtain properties like title, bbox, full xml and links from the metadata record.

In [ ]:
#csw.getrecordbyid(id=['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'])
csw.getrecordbyid(id=[selected_record])

In [ ]:
#rec = csw.records['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE']
rec = csw.records[selected_record]

In [ ]:
selected_record

In [ ]:
rec.title

In [ ]:
rec.xml

In [ ]:
rec.references

In [ ]:
print("dataset bbox = (%s, %s, %s, %s)" % (rec.bbox.miny, rec.bbox.minx, rec.bbox.maxy, rec.bbox.maxx))

Using the [geolinks](https://github.com/geopython/geolinks) Python library we can filter the links that are of a specific type (here WMS and WCS links to be used for visualization)

In [ ]:
msg = 'geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n'.format
for ref in rec.references:
    print(msg(geolink=sniff_link(ref['url']), **ref))

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WMS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WCS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

Finally we demonstrate how to show the record footprint on a map, using the [Folium](https://github.com/python-visualization/folium) Python library:

In [ ]:
centre_x = float(rec.bbox.minx) + ((float(rec.bbox.maxx) - float(rec.bbox.minx))/2)
centre_y = float(rec.bbox.miny) + ((float(rec.bbox.maxy) - float(rec.bbox.miny))/2)
m = folium.Map(location=[centre_y, centre_x], zoom_start=10, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(rec.bbox.miny), float(rec.bbox.minx)], [float(rec.bbox.maxy), float(rec.bbox.maxx)]]).add_to(m)
m

In this part of the demo, the user will use the OpenSearch capability of the system resource catalogue to discover datasets.

In [ ]:
endpoint = system_catalogue_endpoint + '?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'

Here, the OWSLib OpenSearch client is used.

In [ ]:
os = OpenSearch(endpoint)

We use the description object to retrieve service metadata.

In [ ]:
os.description.shortname

In [ ]:
os.description.longname

In [ ]:
os.description.description

In [ ]:
os.description.urls

In [ ]:
os.description.tags

Then we perform a search using atom encoding.

In [ ]:
results = os.search('application/atom+xml')
len(results)

Another posibility is to use the HTTP GET requests directly with a generic client like requests.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
S = requests.Session()

Here, the user asks for the OpenSearch entrypoint template

In [ ]:
R = S.get(url=endpoint)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A GetRecords request (in the context of OpenSearch) is performed

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A collections search is also demonstrated, for Sentinel2 Level 2A results

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:parentIdentifier=sentinel-2-l2a'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can also use OpenSearch EO mathematical notation to filter based on other parameters

In [ ]:
# OpenSearch EO mathematical notation
# n1 equal to field = n1
# {n1,n2,…} equals to field=n1 OR field=n2 OR …
# [n1,n2] equal to n1 <= field <= n2
# [n1,n2[ equals to n1 <= field < n2
# ]n1,n2[ equals to n1 < field < n2
# ]n1,n2] equal to n1 < field  <= n2
# [n1 equals to n1<= field
# ]n1 equals to n1 < field
# n2] equals to field <= n2
# n2[ equals to field < n2

url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:cloudCover=]20'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can get one record using the identifier through the OpenSearch EO API:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&recordids=S2B_MSIL2A_20241120T091209_R050_T34SGH_20241120T115657'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can search using a bbox parameter:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&bbox=23.3,37.8,24.5,38.8'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can search using a time parameter:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&time=2024-11-20/2024-11-21'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Or separate time start/stop parameters:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&start=2024-11-20&stop=2024-11-21'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

### Working with OGC API Records

[OWSLib](https://geopython.github.io/OWSLib) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. In this demo we’ll work with the CSW, WMS and WCS interfaces.

In this part of the demo, the user will use the system level resource catalogue OGC API Records endpoint to discover data datasets.
The `owslib.ogcapi.records` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
from owslib.ogcapi.records import Records
import folium
import json
import requests

In [ ]:
base_domain = "ax3hub-dev.planetek.net"

In [ ]:
system_catalogue_endpoint = f'https://catalogue.{base_domain}'

In [ ]:
w = Records(system_catalogue_endpoint)

In [ ]:
w.url

Conformance classes supported by the OGC API Records server:

In [ ]:
w.conformance()

OpenAPI document of the OGC API Records server:

In [ ]:
w.api()

Collections available on the catalogue:

In [ ]:
w.collections()

In [ ]:
records = w.records()

In [ ]:
len(records)

The user can then specify the collection to search within the catalogue:

In [ ]:
my_catalogue = w.collection('metadata:main')

In [ ]:
my_catalogue['id']

Collection level queryables:

In [ ]:
w.collection_queryables('metadata:main')

Query the catalogue for all records:

In [ ]:
my_catalogue_query = w.collection_items('metadata:main')

In [ ]:
my_catalogue_query['numberMatched']

Metadata of first result:

In [ ]:
my_catalogue_query['features'][0]

In [ ]:
my_catalogue_query['features'][0]['collection']

In [ ]:
my_catalogue_query['features'][0]['properties']

In [ ]:
my_catalogue_query['features'][0]['properties']['platform']

Query the catalogue using filters:

In [ ]:
my_catalogue_query2 = w.collection_items('metadata:main', bbox=['23.3','37.8','24.5','38.8'])

In [ ]:
my_catalogue_query2['numberMatched']

Full query result:

In [ ]:
my_catalogue_query2

Text CQL query:

In [ ]:
my_catalogue_cql_text_query = w.collection_items('metadata:main', filter="identifier LIKE 'S2B_%'")

In [ ]:
my_catalogue_cql_text_query['numberMatched']

In [ ]:
my_catalogue_cql_text_query['features'][0]['id']

JSON CQL query:

In [ ]:
my_catalogue_cql_json_query = w.collection_items('metadata:main', limit=1, cql={'op': '=', 'args': [{ 'property': 'identifier' }, 'S2B_T34SBJ_20240801T093836_L2A-ARD']})

In [ ]:
my_catalogue_cql_json_query['features'][0]['id']

In [ ]:
my_catalogue_query3 = w.collection_items('metadata:main', bbox=['23.3','37.8','24.5','38.8'])

In [ ]:
my_catalogue_query3['numberMatched']

In [ ]:
bbox = my_catalogue_query3['features'][1]['bbox']

In [ ]:
centre_x = float(bbox[0]) + ((float(bbox[2]) - float(bbox[0]))/2)
centre_y = float(bbox[1]) + ((float(bbox[3]) - float(bbox[1]))/2)
m = folium.Map(location=[centre_y, centre_x], zoom_start=7, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(bbox[1]), float(bbox[0])], [float(bbox[3]), float(bbox[2])]]).add_to(m)
m

### Working with pySTAC

In [ ]:
import json
from pystac import Catalog, get_stac_version

In [ ]:
# Read the example catalog
root_catalog = Catalog.from_file('../data/gneo/example-catalog/catalog.json')

In [ ]:
root_catalog.describe()

In [ ]:
# Print some basic metadata from the Catalog
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

In [ ]:
print(get_stac_version())

In [ ]:
collections = list(root_catalog.get_collections())

print(f"Number of collections: {len(collections)}")
print("Collections IDs:")
for collection in collections:
    print(f"- {collection.id}")

In [ ]:
collection = root_catalog.get_child("landsat-8-l1")
if collection is None:
    print("Collection is Empty. Check your downloads and try again.")
else:
    print("Collection has a root child. You may proceed to the following steps.")

In [ ]:
items = list(root_catalog.get_all_items())

print(f"Number of items: {len(items)}")
for item in items:
    print(f"- {item.id}")

In [ ]:
item = root_catalog.get_item("LC80140332018166LGN00", recursive=True)

In [ ]:
item.geometry

In [ ]:
item.bbox

In [ ]:
item.datetime

In [ ]:
item.collection_id

In [ ]:
item.get_collection()

In [ ]:
item.common_metadata.instruments

In [ ]:
item.common_metadata.platform

In [ ]:
item.common_metadata.gsd

In [ ]:
item.stac_extensions

In [ ]:
from pystac.extensions.eo import EOExtension
from pystac.extensions.label import LabelExtension

In [ ]:
EOExtension.has_extension(item)

In [ ]:
LabelExtension.has_extension(item)

In [ ]:
eo_item_ext = EOExtension.ext(item)
eo_item_ext.cloud_cover

In [ ]:
item.properties['eo:cloud_cover']

In [ ]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

In [ ]:
asset = item.assets['B3']
asset.to_dict()

In [ ]:
eo_asset_ext = EOExtension.ext(asset)
bands = eo_asset_ext.bands
bands

In [ ]:
bands[0].to_dict()

### Working with STAC API

In this part of the demo, the user will use the system level resource catalogue STAC API endpoint to discover data datasets.
The `pystac_client` library is used as a STAC client.

In [ ]:
from pystac_client import Client
from pystac_client import ConformanceClasses

In [ ]:
#system_catalogue_endpoint = f'https://catalogue.{base_domain}/stac'
system_catalogue_endpoint = "https://earth-search.aws.element84.com/v1"

In [ ]:
catalog = Client.open(system_catalogue_endpoint)

STAC catalogue metadata:

In [ ]:
catalog.id

In [ ]:
catalog.title

In [ ]:
catalog.description

Conformance classes supported by the STAC API endpoint:

In [ ]:
dir(ConformanceClasses)

Validation of STAC API using `pystac_client`:

In [ ]:
catalog.conforms_to(ConformanceClasses.CORE)

In [ ]:
catalog.conforms_to(ConformanceClasses.ITEM_SEARCH)

Query the catalogue for collections:

In [ ]:
collection_search = client.collection_search(
    q='"sentinel-2" OR "sentinel-1"',
)

In [ ]:
print(f"{collection_search.matched()} collections found")

In [ ]:
for collection in collection_search.collections():
    print(collection.id)

Query the catalogue for all STAC items in one collection:

In [ ]:
COLLECTION_ID = "sentinel-2-l2a"

In [ ]:
mysearch = catalog.search(collections=[COLLECTION_ID], max_items=10)
print(f"{mysearch.matched()} items found")

In [ ]:
for item in mysearch.items():
    print(item)

In [ ]:
items = mysearch.get_items()

In [ ]:
mysearch.item_collection()

Query the STAC API using filters:

In [ ]:
search = client.search(
    max_items=10,
    collections=[COLLECTION_ID],
    bbox=[23.3,37.8,24.5,38.8]
)

In [ ]:
print(f"{search.matched()} items found")

Iterate through the query results:

In [ ]:
for item in search.items():
    print(item.id)

Show first STAC item JSON:

In [ ]:
first_item = next(mysearch.items())
first_item

In [ ]:
item_collection = search.item_collection()
item_collection.save_object('my_itemcollection.json')

### Creating STAC Catalog

In [ ]:
# pip install shapely

In [ ]:
import os
import json
import rasterio
import urllib.request
import pystac
from shapely.geometry import Polygon, mapping
from datetime import datetime, timezone
from tempfile import TemporaryDirectory

In [ ]:
# Set temporary directory to store source data
tmp_dir = TemporaryDirectory()
img_path = os.path.join(tmp_dir.name, 'image.tif')

In [ ]:
url = ("https://eks-1-hub.s3.eu-central-1.amazonaws.com/data/data_ard/SENTINEL-2/S2A_MSIL2A_20240801T084601_N0511_R107_T35SNC_20240801T150054.SAFE/T35SNC_20240801T084601_B02_10m.tif")
urllib.request.urlretrieve(url, img_path)

In [ ]:
catalog = pystac.Catalog(id='tutorial-catalog', description='This catalog is a basic demonstration catalog utilizing a scene from Axis 3 Hub ARD.')

In [ ]:
print(list(catalog.get_children()))
print(list(catalog.get_items()))

In [ ]:
print(json.dumps(catalog.to_dict(), indent=4))

In [ ]:
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [ ]:
# Run the function and print out the results
bbox, footprint = get_bbox_and_footprint(img_path)
print("bbox: ", bbox, "\n")
print("footprint: ", footprint)

In [ ]:
datetime_utc = datetime.now(tz=timezone.utc)

In [ ]:
item = pystac.Item(id='local-image',
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime_utc,
                 properties={})

In [ ]:
print(item.get_parent() is None)

In [ ]:
catalog.add_item(item)

In [ ]:
item.get_parent()

In [ ]:
catalog.describe()

In [ ]:
# Add Asset and all its information to Item 
item.add_asset(
    key='image',
    asset=pystac.Asset(
        href=img_path,
        media_type=pystac.MediaType.GEOTIFF
    )
)

In [ ]:
print(json.dumps(item.to_dict(), indent=4))

In [ ]:
print(catalog.get_self_href() is None)
print(item.get_self_href() is None)

In [ ]:
catalog.normalize_hrefs(os.path.join(tmp_dir.name, "stac"))

In [ ]:
print("Catalog HREF: ", catalog.get_self_href())
print("Item HREF: ", item.get_self_href())

In [ ]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)
#catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)

In [ ]:
with open(catalog.self_href) as f:
    print(f.read())

In [ ]:
with open(item.self_href) as f:
    print(f.read())

In [ ]:
tmp_dir.cleanup()

### Working with ODC-STAC

In [ ]:
# pip install odc-stac

In [ ]:
from odc.stac import configure_rio, stac_load
import yaml

In [ ]:
km2deg = 1.0 / 111
x, y = (113.887, -25.843)  # Center point of a query
r = 100 * km2deg
bbox = (x - r, y - r, x + r, y + r)

In [ ]:
catalog = Client.open("https://earth-search.aws.element84.com/v1")

query = catalog.search(
    collections=["sentinel-2-l2a"], datetime="2021-09-16", limit=100, bbox=bbox
)

items = list(query.items())
print(f"Found: {len(items):d} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.item_collection_as_dict()

In [ ]:
cfg = """---
sentinel-s2-l2a-cogs:
  assets:
    '*':
      data_type: uint16
      nodata: 0
      unit: '1'
    SCL:
      data_type: uint8
      nodata: 0
      unit: '1'
    visual:
      data_type: uint8
      nodata: 0
      unit: '1'
  aliases:  # Alias -> Canonical Name
    red: B04
    green: B03
    blue: B02
"*":
  warnings: ignore # Disable warnings about duplicate common names
"""
cfg = yaml.load(cfg, Loader=yaml.SafeLoader)

In [ ]:
crs = "epsg:3857"

In [ ]:
r = 6.5 * km2deg
small_bbox = (x - r, y - r, x + r, y + r)

xx = stac_load(
    items,
    bands=("red", "green", "blue"),
    crs=crs,
    resolution=10,
    chunks={},  # <-- use Dask
    groupby="solar_day",
    bbox=small_bbox,
)
display(xx)

In [ ]:
xx.odc.geobox

In [ ]:
%%time
xx = xx.compute()

In [ ]:
_ = (
    xx.isel(time=0)
    .to_array("band")
    .plot.imshow(
        col="band",
        size=4,
        vmin=0,
        vmax=4000,
    )
)